In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pymetis
import networkx as nx
import time
from networkx.algorithms import community
from random import shuffle
import math
import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch.nn import init
import pdb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torch_geometric.data import Data
import torch.optim as optim
import pywt
from scipy.stats import norm
import scipy.interpolate as interp
import collections

In [2]:
def read_data(file_path):
    with h5py.File(file_path, 'r') as f:
    # 查看文件中所有的数据集名称
        dataset_names = list(f.keys())
        print("Datasets in the file:", dataset_names)
        data_dict = {}
        for name in dataset_names:
            if f[name].shape == (): 
                data = f[name][()] 
            else:
                data = f[name][:]
        
            data_dict[name] = data
            
        return data_dict

In [3]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [4]:
def save_data_ash5(data, file_path):
    with h5py.File(file_path, 'w') as f:
        keys =list(data.keys())
        for i in range(len(keys)):
            f.create_dataset(keys[i], data=data[keys[i]])

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data_1, data_2, targets):
        self.data_1 = data_1
        self.data_2 = data_2
        self.targets = targets

    def __len__(self):
        return len(self.data_1)

    def __getitem__(self, idx):
        return self.data_1[idx], self.data_2[idx], self.targets[idx]

In [7]:
city_names = ["Barcelona","Antwerp"]
root_path = "D:/ThesisData/processed data/"

In [8]:
#Ant_dict = read_data(root_path + city_names[1] + "/input_target/comm_based_input_target.h5")
Bar_dict = read_data(root_path + city_names[0] + "/input_target/comm_based_input_target.h5")


Datasets in the file: ['period_template', 'target_template', 'trend_template']


In [9]:
#Shuffled indicies
indices = np.random.permutation(Bar_dict['period_template'].shape[0])

In [11]:
def percent_divide(city_name, city_dict, indices, root_path, train_ratio, val_ratio, purpose):
    ratio_dict = dict()
    
    keys = list(city_dict.keys())
    train_size = int(train_ratio * city_dict[keys[0]].shape[0])
    val_size = int(val_ratio * city_dict[keys[0]].shape[0])
    test_size = city_dict[keys[0]].shape[0] - train_size - val_size
    
    for key in keys:
        if purpose == "train":
            ratio_dict[key] = city_dict[key][indices][:train_size, :, :, :]
        elif purpose == "vali":
            ratio_dict[key] = city_dict[key][indices][train_size: train_size+val_size, :, :, :]
        elif purpose == "test":
            ratio_dict[key] = city_dict[key][indices][train_size+val_size: , :, :, :]
            
    save_data_ash5(ratio_dict, root_path + city_name + f"/input_target/{purpose}.h5")
    print("done")

In [15]:
city_name = "Barcelona"
city_dict = Bar_dict
train_ratio, val_ratio = 0.6, 0.2
purpose = 'train'
percent_divide(city_name, city_dict, indices, root_path, train_ratio, val_ratio, purpose)

done
